# 🌀 H-AKORN: Hyperbolic Attention with Kuramoto Oscillators

**Google Colab Version**

Este notebook demonstra o H-AKORN transformer com:
1. ✅ Upload do projeto CGT + H-AKORN
2. ✅ Instalação e verificação
3. ✅ Treinamento e visualizações
4. ✅ Análise de dinâmica de fase

**Author:** Éric Gustavo Reis de Sena
**Date:** January 2026

## 📦 1. Setup: Upload e Instalação

In [ ]:
# @title 1.1 Upload CGT Project com H-AKORN
from google.colab import files
import zipfile
import os

!rm -rf /content/cgt_project
!rm -rf /content/cgt_project_with_hakorn
print('📤 Upload cgt_project_with_hakorn.zip:')

uploaded = files.upload()
for f in uploaded:
    if f.endswith('.zip'):
        with zipfile.ZipFile(f, 'r') as z:
            z.extractall('/content')
        print(f'✅ Extracted: {f}')
        os.remove(f)

# Verificar arquivos essenciais
required = [
    '/content/cgt_project_with_hakorn/src/cgt/geometry/lorentz_hardened.py',
    '/content/cgt_project_with_hakorn/src/cgt/models/hakorn/model.py',
    '/content/cgt_project_with_hakorn/src/cgt/models/hakorn/phase_dynamics.py',
    '/content/cgt_project_with_hakorn/src/cgt/models/hakorn/attention.py',
]

print('\n📋 Verificando arquivos:')
all_ok = True
for p in required:
    exists = os.path.exists(p)
    print(f'{'✅' if exists else '❌'} {p.split('/')[-1]}')
    all_ok = all_ok and exists

if all_ok:
    print('\n🎉 Todos os arquivos encontrados!')
else:
    print('\n❌ Alguns arquivos estão faltando. Verifique o upload.')

In [ ]:
# @title 1.2 Instalar Projeto CGT + H-AKORN
import sys

# Adicionar ao path
sys.path.insert(0, '/content/cgt_project_with_hakorn/src')

# Ou instalar via pip
# %cd /content/cgt_project_with_hakorn
# !pip install -e . -q

print('✅ Path configurado!')
print(f'Python path: {sys.path[0]}')

In [ ]:
# @title 1.3 Verificar Imports
try:
    from cgt.geometry.lorentz_hardened import LorentzSubstrateHardened
    from cgt.models.hakorn import HAKORNTransformer, HAKORNLoss
    from cgt.models.hakorn import KuramotoPhaseEvolution, AdaptiveCoupling
    print('✅ Geometria Lorentz importada')
    print('✅ H-AKORN Transformer importado')
    print('✅ Dinâmica de Kuramoto importada')
    print('\n🎉 Todos os imports funcionaram!')
except ImportError as e:
    print(f'❌ Erro no import: {e}')
    print('Verifique se o upload foi feito corretamente.')

## 🔧 2. Configuração do Ambiente

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Configurar device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'🔧 Device: {device}')
if torch.cuda.is_available():
    print(f'   GPU: {torch.cuda.get_device_name(0)}')
    print(f'   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

## 🧮 3. Demonstração: Dinâmica de Kuramoto

A equação de Kuramoto descreve sincronização de osciladores:

$$\frac{d\theta_i}{dt} = \omega_i + \frac{K}{N} \sum_j A_{ij} \sin(\theta_j - \theta_i)$$

**Order parameter**: $r = |\langle e^{i\theta} \rangle| \in [0,1]$

In [ ]:
# Visualizar evolução de fase
from cgt.models.hakorn import KuramotoPhaseEvolution

def demo_kuramoto(num_heads=8, num_steps=100, coupling_strength=2.0):
    phase_module = KuramotoPhaseEvolution(
        d_model=768,
        num_heads=num_heads,
        coupling_strength=coupling_strength,
    )
    
    # Coupling matrix uniforme
    coupling = torch.ones(num_heads, num_heads) / num_heads
    
    # Evoluir fases
    phases_history = []
    order_params = []
    
    for _ in range(num_steps):
        phases, r = phase_module(coupling, batch_size=1)
        phases_history.append(phases[0].detach().numpy())
        order_params.append(r[0].item())
    
    # Plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))
    
    # Evolução de fases
    phases_array = np.array(phases_history)
    for i in range(num_heads):
        ax1.plot(phases_array[:, i], label=f'Head {i+1}')
    ax1.set_xlabel('Step')
    ax1.set_ylabel('Phase θ (rad)')
    ax1.set_title('Phase Evolution')
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax1.grid(True, alpha=0.3)
    
    # Order parameter
    ax2.plot(order_params, linewidth=2)
    ax2.axhline(y=0.5, color='r', linestyle='--', alpha=0.5, label='r=0.5')
    ax2.axhline(y=0.8, color='g', linestyle='--', alpha=0.5, label='r=0.8')
    ax2.set_xlabel('Step')
    ax2.set_ylabel('Order Parameter r')
    ax2.set_title('Synchronization (r)')
    ax2.set_ylim([0, 1])
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f'Final order parameter: r = {order_params[-1]:.3f}')
    print(f'Status: {'Synchronized' if order_params[-1] > 0.7 else 'Partially synchronized' if order_params[-1] > 0.4 else 'Unsynchronized'}')

# Executar demo
demo_kuramoto(num_heads=8, num_steps=100, coupling_strength=2.0)